<a href="https://colab.research.google.com/github/bermanlabemory/gait_signatures/blob/main/Gait_Signatures_Script_4_Short_TimeSelf_Driven_Signatures.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# This script computes the short-time self-driven signatures
Short-time self-driven gait signatures by integrating the model forward in time on full gait cycle, initialized from the internal and kinematic states at the start of the gait cycle.
____________
____________
**NOTE**: This file was ONLY used to select model hyperparameters and generate supplementary figures.

First, Scripts 1-3 must be run to generate the models that are analyzed here.
____________
____________
**Steps:** 
1. Load RNN dynamics model
2. Load phase-averaged externally-signatures
3. Load phase esimates and gait kinematics
3. Idenitfy initial conditions for simulations: the start of each cycle and extract kinematic and internal states. 
____________
4. Using the above initial conditions integrate the model forward in time 1 stride 
  6. Compute error relative to externally-driven prediction
5. Average errors across strides for each trial  
8. Take average & SD error across trials

____________
____________

**Created by**: Michael C. Rosenberg

**Date**: 09/22/22

**Step 0**: Mount (connect to) your google drive folder where you want to save the simulation results and model parameters.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
#drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [ ]:
# check python version 
from platform import python_version
print(python_version())

# check tensorflow version
import tensorflow as tf
print(tf.__version__)

# Check memory
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

3.7.15


**Step 1**: Import necessary packages and functions to develop model

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.python.keras.layers.recurrent import LSTM
from sklearn.model_selection import train_test_split
import sklearn.model_selection as model_selection
import matplotlib.pyplot as plt
import math
import keras as k
import pandas as pd
import numpy as np
from copy import copy
import scipy.io
from sklearn.decomposition import PCA

from scipy.signal import find_peaks
from scipy import interpolate
from numpy import sin,cos,pi,array,linspace,cumsum,asarray,dot,ones
from pylab import plot, legend, axis, show, randint, randn, std,lstsq

from sklearn.manifold import MDS
import csv
import os
from tqdm import tqdm 

In [ ]:
%cd /content/drive/My Drive/NeuromechanicsLab/GaitSignatures/


# Ensure fourierseries.py is in the pathway
!ls -l fourierseries.py

/content/drive/My Drive/NeuromechanicsLab/GaitSignatures
-rw------- 1 root root 7259 May 29  2020 fourierseries.py


In [ ]:
# Phaser wrapper to estimate system phase
import fourierseries
import util
import phaser
import dataloader
# Preprocess data for a single subject - to be send to modeling frameworks
def find_phase(k):
    """
    Detrend and compute the phase estimate using Phaser
    INPUT:
      k -- dataframe
    OUTPUT:
      k -- dataframe
    """
    #l = ['hip_flexion_l','hip_flexion_r'] # Phase variables = hip flexion angles
    y = np.array(k)
    print(y.shape)
    y = util.detrend(y.T).T
    print(y.shape)
    phsr = phaser.Phaser(y=y)
    k[:] = phsr.phaserEval(y)[0,:]
    return k

**Step 2**: Load module in Google Drive

In [ ]:
# The path to save the models and read data from
path = '/content/drive/My Drive/NeuromechanicsLab/GaitSignatures/'

# Insert the directory
import sys
sys.path.insert(0,path)

**Step 3**: Load in data and specify variables/parameters

In [ ]:
# Non-changing variables 

# number of trials in dataset 
trialnum = 72 # 72 total trials

# number of samples in each trial
trialsamp = 1500

# number of features collected per trial
feats = 6

#Batch size - same as the number of traintrials
batch_size = trialnum

# Number of Layers
numlayers = 1

# Choose the number of iterations to train the model- if this script has been run previously enter a value greater than was 
# inputted before and rerun the script. 
finalepoch = 10000

# load the input data/kinematics
datafilepath = path + 'PareticvsNonP_RNNData.csv' #input data
all_csvnp = np.loadtxt(datafilepath,delimiter=',').T

# reshape all the input data into a tensor
all_inputdata_s = all_csvnp.reshape(trialnum,trialsamp,feats) 
csvnp = all_inputdata_s
print('original input data shape is: '+ str(all_csvnp.shape ))
print('input data reshaped is: '+ str(all_inputdata_s.shape))

original input data shape is: (108000, 6)
input data reshaped is: (72, 1500, 6)


**Step 4**: Develop list of model architectures and corresponding variables to train. This step also generates list of folder names and pathways where the models will be saved and accessed later.

In [ ]:
# generate a list of models and corresponding parameters to test 
test_model_nodes = [128, 256,512,1024] 
seqs = [249,499,749] #lookback parameter

test_model_nodes = [ 512] 
seqs = [499] #lookback parameter

# run multiple model architechtures many times to test stability of cost function outputs
runs = 1 # Number of times to train recurrent neural network (RNN) models, starting from random initial conditions. We used this for hyperparameter selection
test_model_seq = np.repeat(seqs, runs) # Specify each model's hyperparameters

count = np.arange(runs)

All_nodes = np.empty([0,1], dtype='int')
All_seq = np.empty([0,1],dtype='int')
All_valseg = np.empty([0,1],dtype='int')
All_trainseg = np.empty([0,1],dtype='int')
All_modelname = []
All_mod_name = []
count = np.empty([0,1],dtype='int'); #initialize model run -- this serves as the model run ID number
ct = 0
for a in test_model_nodes:
  for b in test_model_seq:
    count = np.append(count,  ct + 1 )
    #if statement for valseg, trainseg based on sequence length
    if int(b) == 249:
      trainseg = 4
      valseg = 2
    elif int(b) == 499: 
      trainseg = 2
      valseg = 1
    elif int(b) == 749:
      trainseg = 1
      valseg = 1

    # Store resulting model structures and training plans
    All_nodes = np.append(All_nodes, a) 
    All_seq = np.append(All_seq, int(b))
    All_valseg = np.append(All_valseg, valseg)
    All_trainseg = np.append(All_trainseg, trainseg)
    All_modelname = np.append(All_modelname, 'UNIT_' + str(a) + '_LB_' + str(b) + '_run_' + str(count[-1]) + '/' )
    All_mod_name = np.append(All_mod_name, 'UNIT_' + str(a) + '_LB_' + str(b) + '_run_' + str(count[-1]) )

    if ct+1 < runs:
      ct += 1
    else:
      ct = 0

['UNIT_512_LB_499_run_1' 'UNIT_512_LB_499_run_2' 'UNIT_512_LB_499_run_3'
 'UNIT_512_LB_499_run_4' 'UNIT_512_LB_499_run_5' 'UNIT_512_LB_499_run_6'
 'UNIT_512_LB_499_run_7' 'UNIT_512_LB_499_run_8' 'UNIT_512_LB_499_run_9'
 'UNIT_512_LB_499_run_10']
/content/drive/My Drive/NeuromechanicsLab/GaitSignatures/UNIT_512_LB_499_run_1/UNIT_512_LB_499_run_1_bestwhole.h5


**Step 5**: Generate self-driven short-time gait singatures

1. Load trained model architectures in loop
2. Load phase estimates for the corresponding long-term predictions
3. Load ext. driven H & C values
3. Find cycle trainsitions (2pi -> 0 rads)
4. From each phase transition, use the ext-driven H & C values set the model's initiial internal state
5. Initialize a simulation and self-drive for 200 samples



In [ ]:
for j in range(3,len(All_mod_name)):
  # 1. make folder to store model
  newfoldpath = path + All_mod_name[j]

  # extract path to store each model and generated data
  savepath = path + All_modelname[j]
  mod_name = All_mod_name[j]
  print('Working on: ' + mod_name + ' model ' + str(j) + ' / ' + str(len(All_mod_name)))

  # Specify variables for model run instance
  # Number of Units
  numunits = All_nodes[j]

  # Lookback parameter
  lookback = All_seq[j]

  # Training and Validation Set-up
  trainseg = All_trainseg[j]
  valseg = All_valseg[j]

  # Initialize a signle figure
  plt.figure(figsize = [50,25])
  plotVar = 2

  # Load model and set weights
  with tf.device('/device:GPU:0'):
      model = tf.keras.models.load_model(savepath + mod_name + '_bestwhole.h5') 

      # Set the trained model weights to new model to evaluate external predictions and self-driving 
      model2 = tf.keras.models.Sequential()
      model2.add(tf.compat.v1.keras.layers.CuDNNLSTM(units = numunits, stateful=True, return_sequences=True, batch_input_shape =(1,None,feats)))
      model2.add(tf.compat.v1.keras.layers.Dense(units=6))
      model2.compile(loss='mse', optimizer='adam',metrics=['accuracy'])

      # Set model 2 weights from model 1
      for l1, l2 in zip(model.layers, model2.layers):
        assert l1.get_config().keys()==l2.get_config().keys()
      for key, v in l1.get_config().items():
          if key=='name':
              continue
          if not v==l2.get_config()[key]:
              print(l1.name, key, 'not matching.', 'Old,new : ', v, l2.get_config()[key], '\n')

      model2.set_weights(model.get_weights())
      assert np.all([np.array_equal(w1, w2) for (w1,w2) in zip(model.get_weights(), model2.get_weights())]) 

  # 2. Load phase data
  phs = scipy.io.loadmat(savepath + mod_name + '_PhaseVariables.mat')
  phs = phs['PhaseVariables'] # [num trials x num samples per trial]

  # 3. Load Ext Driven H & C values
  temp = scipy.io.loadmat(savepath + mod_name + '_extdriveHCs_testset.mat')
  temp = temp['ext_drive_sigs']
  ExternalDriveHCs = np.reshape(temp, (trialnum,trialsamp,numunits*2))  # [num trials x num samples per trial x num-nodes-times-2]

  # 4. Find initial states and corresponding H & C values
  M = len(phs) # number of trials
  print(M)
  trialConcatLen = 1000; # concatentate predictions to this trial length to ensure that we evaluate the same amount of data across trials

  # 5. Preallocate arrays for simulations
  HC_self_concat = np.empty(shape=[0, numunits*2]) # Self driven sigs
  HC_ext_concat = np.empty(shape=[0, numunits*2]) # ExternallyDrivenSigs for comparison
  DATA_concat = np.empty(shape=[0, feats]) # Data
  Pred_concat = np.empty(shape=[0, feats]) # Predictions
  Predicted_KIN = np.empty(shape = [trialnum, trialsamp, feats]) # We don't simulate for the full 1500 samples, but we're OK with NaNs at the end
  test_ind = np.arange(0, len(all_inputdata_s), 1)  # run all input data through
  R2_muSD = np.empty(shape=[0,2]) # R-squared

  # For each trial, get initial H & C values, and simulate from each initial condition
  for mm in range(M):
    R2 = np.empty(shape=[0]) # R-squared
    # Holds data for each trial
    HC_self_concat_tr = np.empty(shape=[0, numunits*2]) # Self driven sigs
    HC_ext_concat_tr = np.empty(shape=[0, numunits*2]) # ExternallyDrivenSigs for comparison

    print('Generating short-time predictions for trial: ' + str(mm+1) + ' of ' + str(len(test_ind)))
    # Get transitions
    transitionInds,_ = find_peaks(phs[mm], prominence = np.pi*3/2)
    Nx = len(transitionInds)-1 # Number of initial conditions minus one

    # Get initial states
    X0 = all_inputdata_s[mm,transitionInds,:]

    # Get H & C values
    HC0 = ExternalDriveHCs[mm,transitionInds,:]

    # Plot as sanity check
    '''plt.figure()
    plt.plot(phs[mm], label = 'Phase')
    plt.plot(transitionInds,phs[mm][transitionInds],'*', label = 'Initial conditions')
    plt.xlabel('Sample')
    plt.ylabel('Phase (rad)')
    plt.legend()
    plt.show()'''

    # Pre-allocate arrays
    preds = np.zeros((1, trialsamp, feats))
    hcs = np.zeros((2, trialsamp, numunits))
    data = np.zeros((1, trialsamp, feats))
    extD = np.zeros((1, trialsamp, numunits*2))

    if Nx < 20 and Nx > 5:
      for g in range(Nx): # For each initial state, reset model's internal state and simulate
        with tf.device('/device:GPU:0'):
          print('Simiulating cycle: ' + str(g+1) + ' of ' + str(Nx))
          # Feed in externally-driven H's and C's at each initial condition
          model2.layers[0].reset_states( [np.reshape(HC0[g,0:numunits], (1,numunits)), np.reshape(HC0[g,numunits:], (1,numunits))] )          

          # Integrate
          sampleWindow = transitionInds[g+1] - transitionInds[g] # prediction length
          windowInds = range(transitionInds[g], transitionInds[g+1]) # prediction horizon - integrate for this many samples
          for i in tqdm(windowInds):
              data[:,i,:] = all_inputdata_s[mm,i,:]
              extD[:,i,:] = ExternalDriveHCs_3[mm,i,:]
              if i == transitionInds[g]:
                  preds[:,i,:] = all_inputdata_s[mm,i,:][None,None,:] # Initialize the simulation
              else:
                  preds[:,i,:] = model2.predict_step(preds[0,i-1,:][None,None,:]) # Integrate forward in time

              # Extract internal states
              hcs[:,i,:] = np.array(model2.layers[0].states)[:,0]
              Hvalues = hcs[0][windowInds,:]
              Cvalues = np.tanh(hcs[1][windowInds,:])
              HC_self = np.concatenate((Hvalues, Cvalues), axis=1) # concatenate Hs and Cs of single trial

          # concatenate trials
          HC_ext_concat_tr = np.append(HC_ext_concat_tr, extD[0,windowInds,:], axis = 0) # Internal states
          HC_self_concat_tr = np.append(HC_self_concat_tr, HC_self, axis = 0) # Internal states
          DATA_concat = np.append(DATA_concat, data[0][windowInds,:], axis = 0) # Data
          Pred_concat = np.append(Pred_concat, preds[0][windowInds,:], axis = 0) # Predictions

          # Compute prediction accuracy across trials
          fullInds = range(transitionInds[0], transitionInds[-1])
          X = np.reshape(data[0][fullInds,:],(-1,1))
          X_hat = np.reshape(preds[0][fullInds,:],(-1,1))
          num = np.sum((X-X_hat)**2)
          den = np.sum((X-np.mean(X))**2)
          # Compute and store R-squared value
          R2 = np.append(R2,[1-num/den], axis = 0)

      # Store predicted internal states
      HC_ext_concat = np.append(HC_ext_concat, HC_ext_concat_tr[0:trialConcatLen,:], axis = 0) # Extenrally-driven
      HC_self_concat = np.append(HC_self_concat, HC_self_concat_tr[0:trialConcatLen,:], axis = 0) # Self-driven

      # Plot data and predictions for each trial
      plt.subplot(8,9,mm+1)
      plt.plot(all_inputdata_s[mm,:,plotVar], label = 'data')
      plt.plot(preds[0,:,plotVar], label = 'prediction')    
      plt.title('Trial' + str(mm+1))
      if (mm+1) % 9 == 1:
        plt.ylabel('Joint angle (deg)')
      if mm > 62:
        plt.xlabel('Sample')      
      if mm == 0:
        plt.legend()

    else: # Case where we identify too many or too few trials. Provide a trivial R-squared value that we conver to NaN during analysis
      
      print('Failure! Trial ' + str(mm+1))
      data[:,:,:] = all_inputdata_s[mm,:,:]
      extD = np.zeros((trialConcatLen, 2*numunits)) # concatenate Hs and Cs of single trial
      HC_ext_concat = np.append(HC_ext_concat, extD, axis = 0) # concatenate trials
      HC_self = np.zeros((trialConcatLen, 2*numunits)) # concatenate Hs and Cs of single trial
      HC_self_concat = np.append(HC_self_concat, HC_self, axis = 0) # concatenate trials
      DATA_concat = np.append(DATA_concat, data[0], axis = 0) # Data
      R2 = 0.001*np.array([1,1,1,1,1])
    
    # Store average and SD of R2
    aa = np.array([np.mean(R2), np.std(R2)])
    R2_muSD = np.append(R2_muSD, aa[np.newaxis,:], axis = 0)

  # Same figure
  plt.savefig(savepath + mod_name + 'selfDriveVsData.png', dpi = 150)
  plt.show()   
    

  # Save resulting HCs and R2 corresponding to kinematics predictions
  scipy.io.savemat(savepath + mod_name + '_selfExtDriveHC_shortTime.mat', {'HC_self_concat': HC_self_concat, 'HC_ext_concat': HC_ext_concat, 'R2_muSD': R2_muSD})
